In [ ]:
import pygame
from pygame.locals import *

pygame.init()

#setting a timer so that the game runs at the same speed in every PC
clock = pygame.time.Clock()
#setting frames per second
fps = 60

screen_width = 1000
screen_height = 600

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Platformer")

#define game variables
tile_size = 50
game_over = 0

#load images
bg_image = pygame.image.load('sky.jpg')
sun_image = pygame.image.load('sun.png')
restart_img = pygame.image.load('restart.png')

def draw_grid():
    for line in range(0, 20):
        pygame.draw.line(screen, (255, 255, 255), (0, line * tile_size), (screen_width, line * tile_size))
        pygame.draw.line(screen, (255, 255, 255), (line * tile_size, 0), (line * tile_size, screen_height))
        
class Button():
    def __init__(self,x,y,image):
        self.image = image
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        
    def draw(self):
        #draw button
        pygame.blit( self.image, self.rect )
        
        
class Player():
    def __init__(self, x, y):
        self.images_right = []
        self.images_left = []
        self.index = 0
        self.counter = 0
        #this will load all the images for the iteration
        for num in range(1, 7):
            img_right = pygame.image.load(f'player{num}.png')
            img_right = pygame.transform.scale(img_right, (40, 80))
            img_left = pygame.transform.flip(img_right, True, False)
            self.images_right.append(img_right)
            self.images_left.append(img_left)
                                             
        #it will initialze the image with zero index
        self.image = self.images_right[self.index]
        self.dead_image = pygame.image.load('ghost.png')
        #this rectangle is a property of the player instance we created
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.width = self.image.get_width()
        self.height = self.image.get_height()
        self.y_velocity = 0
        self.jumped = False
        self.direction = 0
        

    def update(self, game_over):
        
        #For keypresses
        dx = 0
        dy = 0
        walk_smooth = 10
        
        if game_over==0:
            key = pygame.key.get_pressed()
            if key[pygame.K_SPACE] and self.jumped == False:
                self.y_velocity = -15
                self.jumped = True
            if key[pygame.K_SPACE] == False:
                self.jumped = False
            if key[pygame.K_LEFT]:
                dx -= 2
                self.counter+=1
                self.direction = -1                          
            if key[pygame.K_RIGHT]:
                dx += 2
                self.counter+=1
                self.direction = 1
            #we want animation to stop immediately when we release the keys
            if key[pygame.K_LEFT] == False and key[pygame.K_RIGHT] == False:
                self.counter = 0
                self.index = 0
                self.image = self.images_right[self.index]
                if self.direction == 1:
                    self.image = self.images_right[self.index]
                if self.direction == -1:
                    self.image = self.images_left[self.index]

            #handle animation
            if self.counter > walk_smooth:
                self.counter = 0
                self.index +=1
                if self.index >= len(self.images_right):
                    self.index = 0
                if self.direction == 1:
                    self.image = self.images_right[self.index]
                if self.direction == -1:
                    self.image = self.images_left[self.index]

            #add gravity
            self.y_velocity +=1
            if self.y_velocity > 10:
                self.y_velocity = 10

            dy += self.y_velocity

            #check for collisions
            #we will apply a check for each tile in our data
            #check for collision in both x and y direction
            #check for collision in y direction
            #we will check for rect collision bcz all of our objects are rectangles
            #this check will tell us about the collision after the 
            #player has already overlapped with the tile

            for tile in world.tile_list:
                #check for collision in x direction
                if tile[1].colliderect(self.rect.x + dx, self.rect.y, self.width, self.height ): 
                    dx = 0 #stop moving
                #check for collision in y direction
                if tile[1].colliderect(self.rect.x, self.rect.y + dy, self.width, self.height ): 
                    #check if below the ground i.e. jumping
                    #+ve velocity means falling due to gravity -ve velocity means jimping
                        if self.y_velocity < 0:
                            dy = tile[1].bottom - self.rect.top #moves till the little distance he can before collision
                            self.y_velocity = 0
                        elif self.y_velocity >= 0: #falling or landing on something
                                dy = tile[1].top - self.rect.bottom
                                self.y_velocity = 0

            #collision checks with enemies
            if pygame.sprite.spritecollide(self, enemy_group, False):  #self is player itself
                game_over = -1  #if the collision happens set value of game over equals -1

            #collision checks with lava
            if pygame.sprite.spritecollide(self, lava_group, False): 
                game_over = -1  


            #dx and dy don't immediately update the x and y coordinates
            #so we will use them for collision detection
            #so that we will know about the collision before overlapping
            #so we are checking for collision along x and y axis before we make any change along these coordinates
            #dx and dy tells that player hasn't already moved but he intends to move this much
            #update player coordinates
            self.rect.x +=dx
            self.rect.y +=dy

        #player doesn't fall out of the screen
        elif game_over == -1:
            self.image = self.dead_image
            if self.rect.y >200:
                self.rect.y -=5
          
        if self.rect.bottom > screen_height:
            self.rect.bottom = screen_height
            dy = 0
        
        #draw player on the screen
        screen.blit(self.image, self.rect)
        pygame.draw.rect( screen, (255,255,255), self.rect, 2)
        
        return game_over
        
class World():
    def __init__(self, data):
        self.tile_list = []

        #load images
        dirt_img = pygame.image.load('dirt.png')
        grass_img = pygame.image.load('grass.png')

        row_count = 0
        for row in data:
            col_count = 0
            for tile in row:
                #if tile is 1 create dirt
                if tile == 1:
                    img = pygame.transform.scale(dirt_img, (tile_size, tile_size))
                    img_rect = img.get_rect()
                    img_rect.x = col_count * tile_size
                    img_rect.y = row_count * tile_size
                    tile = (img, img_rect)
                    self.tile_list.append(tile)
                #if tile is 2 create grass
                if tile == 2:
                    img = pygame.transform.scale(grass_img, (tile_size, tile_size))
                    img_rect = img.get_rect()
                    img_rect.x = col_count * tile_size
                    img_rect.y = row_count * tile_size
                    tile = (img, img_rect)
                    self.tile_list.append(tile)
                if tile == 3:
                    enemy = Enemy(col_count * tile_size, row_count * tile_size + 15)
                    enemy_group.add(enemy)
                #lava
                if tile == 6:
                    lava = Lava(col_count * tile_size, row_count * tile_size + (tile_size//2)) # x and y coordinates
                    lava_group.add(lava)
                col_count += 1
            row_count += 1

    def draw(self):
        for tile in self.tile_list:
            screen.blit(tile[0], tile[1])
            pygame.draw.rect(screen,(255,255,255), tile[1], 2)

#inheritance Sprite class is already built in
#enemy class is a child of this built in sprite class
#it has draw and update already built in

class Enemy(pygame.sprite.Sprite): 
    def __init__(self,x,y):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load('enemy0.png') #load image
        self.rect = self.image.get_rect() #get rect fot that image
        self.rect.x = x
        self.rect.y = y #set x and y axis to position it correctly
        self.move_direction = 1
        self.move_counter = 0
        
    #we will override the update fn which is at initial empty
    
    def update(self):
        #moving them in x direction
        self.rect.x += self.move_direction
        # a counter is also increasing 
        #we will check if the counter reaches a certain limit it will flip its direction
        self.move_counter +=1
        if self.move_counter > 30:
            self.move_direction *= -1
            #it makes the counter -51 and then it adds one into it untill it reaches zero
            #in this way it flips its direction in the opposite way
            self.move_counter *= -1
            
class Lava(pygame.sprite.Sprite):
    def __init__(self, x, y):
        pygame.sprite.Sprite.__init__(self)
        img = pygame.image.load('lavaa.png') #load images
        self.image = pygame.transform.scale(img, (tile_size, tile_size))  #changes made here
        self.rect = self.image.get_rect() #get rect fot that image
        self.rect.x = x
        self.rect.y = y #set x and y axis to position it correctly    
            

world_data = [
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], #0 is nothing
[1, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 1], #1 is dirt
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 2, 2, 1], #2 is grass
#[1, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 7, 0, 5, 0, 0, 0, 1],# 3 for enemies
#[1, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 1], 
#[1, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 
#[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 
[1, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 7, 0, 0, 7, 2, 2, 2, 0, 1], 
#[1, 0, 2, 0, 0, 7, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 
[1, 0, 0, 0, 0, 0, 4, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1],  #enemy here 
[1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 3, 0, 1], 
#[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 7, 0, 0, 0, 2, 2, 2, 1], 
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 1], 
[1, 0, 0, 0, 0, 0, 2, 2, 2, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1], 
#[1, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
#[1, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
[1, 1, 1,1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]

player = Player(100, screen_height - 130)
#creating an empty group for enemy sprites just like we created an empty list for tiles
enemy_group = pygame.sprite.Group()
lava_group = pygame.sprite.Group()

world = World(world_data)

run = True
while run:
    
    clock.tick(fps)  
    screen.blit(bg_image, (0,0))
    screen.blit(sun_image, (100,100))
    
    world.draw()
    #create button
    restart_button = Button( screen_width //2 -50, screen_height//2 +100, restart_img)
    #drawing enemies on the screen
    if game_over==0:
        enemy_group.update() #only update enemies movement untill player hits the lava or the enemy
        
    enemy_group.draw(screen)
    #drawing lava on screen
    lava_group.draw(screen)
    
    game_over = player.update(game_over)
    #draw_grid()
    
    #if player has died
    if game_over== -1:
        restart_button.draw()
    for event in pygame.event.get(): 
        if event.type == pygame.QUIT: 
            run = False
            
    pygame.display.update()
    
pygame.quit()